In [10]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [11]:
loader=PyPDFDirectoryLoader("pdfs")

In [12]:
data=loader.load()

In [13]:
#Tokenization
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)


In [14]:
texts=text_splitter.split_documents(data)

In [ ]:
print(texts[1].page_content)

In [16]:
len(texts)

91

In [17]:
import os
os.environ['OPENAI_API_KEY']=""

In [ ]:
embeddings=OpenAIEmbeddings()

In [ ]:
embeddings.embed_query("How are you")

In [ ]:
len(embeddings.embed_query("How are you"))

1536

In [19]:
import os
import pinecone
# import os
# os.environ['PINECONE_API_KEY'] = ''

# PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY", '')
# PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV", 'gcp-starter')

# pinecone_instance = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)


/home/seaflux/VectorDB/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [20]:
index_name = "testing" # put in the name of your pinecone index here

In [22]:
index = pinecone.Index(name='testing', api_key='', host='')


PineconeConfigurationError: You haven't specified an Api-Key.

In [ ]:
#Create Embeddings for each of the Text Chunk

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [ ]:
docsearch

In [ ]:
docsearch.as_retriever()





VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7fda328e1250>)

In [ ]:
query="what is the meaning of multihead attention"

In [ ]:
docs = docsearch.similarity_search(query)

In [ ]:
docs

[Document(page_content='i∈Rdmodel×dk,WV\ni∈Rdmodel×dv\nandWO∈Rhdv×dmodel.\nIn this work we employ h= 8 parallel attention layers, or heads. For each of these we use\ndk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost\nis similar to that of single-head attention with full dimensionality.\n3.2.3 Applications of Attention in our Model\nThe Transformer uses multi-head attention in three different ways:'),
 Document(page_content='output values. These are concatenated and once again projected, resulting in the final values, as\ndepicted in Figure 2.\nMulti-head attention allows the model to jointly attend to information from different representation\nsubspaces at different positions. With a single attention head, averaging inhibits this.\nMultiHead( Q, K, V ) = Concat(head 1, ...,head h)WO\nwhere head i= Attention( QWQ\ni, KWK\ni, V WV\ni)\nWhere the projections are parameter matrices WQ\ni∈Rdmodel×dk,WK\ni∈Rdmodel×dk,WV'),
 Document(page_content='Sc

In [ ]:
llm=OpenAI()

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [ ]:
query = "multihead attention"

In [ ]:
qa.invoke(query)

{'query': 'multihead attention',
 'result': ' Multi-head attention is a technique used in the Transformer model to jointly attend to information from different representation subspaces at different positions. It involves using multiple parallel attention layers, or "heads," and linearly projecting the queries, keys, and values to different dimensions before combining them to produce the final output values. This allows the model to attend to different aspects of the input data simultaneously, improving overall performance.'}

In [ ]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Answer:  Transformers is a sequence transduction model based entirely on attention, replacing the recurrent layers commonly used in encoder-decoder architectures with multi-headed self-attention. It was proposed as a new simple network architecture for translation tasks and has shown to be significantly faster to train than other architectures.
Answer:  The authors of this research paper are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, and Aidan N. Gomez. 
Exiting


SystemExit: 

/home/seaflux/Pinecone_VectorDB/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
